# Basic Code

In [1]:
from csb_multi_env import *

2024-05-13 00:09:08,508	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-13 00:09:09,403	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
env = CodersStrikeBackMulti(dt=1)

In [3]:
env.reset()

({'car_0_0': array([ 3.00000000e+00, -2.99419829e+00,  1.40498962e+04,  4.97482494e+03,
          0.00000000e+00,  0.00000000e+00,  6.68000000e+02,  2.98800000e+03,
          4.23900000e+03,  7.24900000e+03,  1.41940000e+04,  7.31200000e+03,
          1.00300000e+04,  1.72800000e+03,  1.40750000e+04,  3.17500000e+03]),
  'car_1_0': array([ 3.00000000e+00, -3.08292706e+00,  1.40666321e+04,  3.77494165e+03,
          0.00000000e+00,  0.00000000e+00,  6.68000000e+02,  2.98800000e+03,
          4.23900000e+03,  7.24900000e+03,  1.41940000e+04,  7.31200000e+03,
          1.00300000e+04,  1.72800000e+03,  1.40750000e+04,  3.17500000e+03]),
  'car_0_1': array([3.00000000e+00, 3.11082113e+00, 1.40833679e+04, 2.57505835e+03,
         0.00000000e+00, 0.00000000e+00, 6.68000000e+02, 2.98800000e+03,
         4.23900000e+03, 7.24900000e+03, 1.41940000e+04, 7.31200000e+03,
         1.00300000e+04, 1.72800000e+03, 1.40750000e+04, 3.17500000e+03]),
  'car_1_1': array([3.00000000e+00, 3.02209236e+00, 1

In [4]:
env.reset()
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(env.action_space.sample())
    done = done['__all__']
    rewards.append(reward)

In [5]:
rewards[-1]

{'car_0_0': -1, 'car_1_0': 1, 'car_0_1': -1, 'car_1_1': 1}

In [6]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [7]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=4)

2024-05-13 00:09:11,604	INFO worker.py:1740 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.14
Ray version:,2.21.0
Dashboard:,http://127.0.0.1:8265


In [8]:
def policy_mapping_fn(agent_id, episode, worker=None, **kwargs):
    # At first, only have main play against the random main exploiter.
    return "main" if episode.episode_id % 2 == agent_id else "main_exploiter_0"

def agent_to_module_mapping_fn(agent_id, episode, **kwargs):
    # At first, only have main play against the random main exploiter.
    return "main" if hash(episode.id_) % 2 == agent_id else "main_exploiter_0"

def _get_multi_agent():
    names = {
        # Our main policy, we'd like to optimize.
        "main",
        # First frozen version of main (after we reach n% win-rate).
        "main_0",
        # Initial main exploiters (one random, one trainable).
        "main_exploiter_0",
        "main_exploiter_1",
        # Initial league exploiters (one random, one trainable).
        "league_exploiter_0",
        "league_exploiter_1",
    }
    if args.enable_new_api_stack:
        policies = names
        spec = {
            mid: SingleAgentRLModuleSpec(
                module_class=(
                    RandomRLModule
                    if mid in ["main_exploiter_0", "league_exploiter_0"]
                    else None
                )
            )
            for mid in names
        }
    else:
        policies = {
            mid: PolicySpec(
                policy_class=(
                    RandomPolicy
                    if mid in ["main_exploiter_0", "league_exploiter_0"]
                    else None
                )
            )
            for mid in names
        }
        spec = None
    return {"policies": policies, "spec": spec}

In [9]:
agent_ids = CodersStrikeBackMulti()._agent_ids
sym_policies = {agent_id: f"policy_{agent_ids[0]}" for agent_id in agent_ids}  # Symmetric
asym_policies = {agent_id: f"policy_{agent_id}" for agent_id in agent_ids}  # Asymmetric

def policy_mapping_fn(agent_id, episode, worker, *, policies=None, **kwargs):
    return policies[agent_id]


policies = sym_policies
policy_mapping = partial(policy_mapping_fn, policies=policies)

config = (
        PPOConfig()
        .environment(CodersStrikeBackMulti)
        .framework('torch')
        .training(train_batch_size=1024, gamma=0.99)
        .resources(num_gpus=0)
        .rollouts(num_rollout_workers=3)
        .multi_agent(policies=list(policies.values()),
                     policy_mapping_fn=policy_mapping)
    )

stop = {"timesteps_total": 120}

# THE WAY OF CODING IS A BIT DIFFERENT HERE FROM THE SINGLE AGENT ONE.
tuner = tune.Tuner(
        "PPO",
        param_space=config.to_dict(),
        run_config=air.RunConfig(stop=stop),
    )

res = tuner.fit()
best_result = res.get_best_result(metric="loss", mode="min")
best_checkpoint = best_result.checkpoint


(PPO pid=7002) 2024-05-13 00:09:14,721	WARNING deprecation.py:50 -- DeprecationWarning: `WorkerSet(num_workers=... OR local_worker=...)` has been deprecated. Use `EnvRunnerGroup(num_env_runners=... AND local_env_runner=...)` instead. This will raise an error in the future!
(PPO pid=7002) 2024-05-13 00:09:14,721	WARNING deprecation.py:50 -- DeprecationWarning: `max_num_worker_restarts` has been deprecated. Use `AlgorithmConfig.max_num_env_runner_restarts` instead. This will raise an error in the future!
(PPO pid=7002) Install gputil for GPU system monitoring.
(PPO pid=7002) 2024-05-13 00:09:17,323	ERROR actor_manager.py:519 -- Ray error, taking actor 1 out of service. ray::RolloutWorker.apply() (pid=7003, ip=127.0.0.1, actor_id=1fbfbbb68e579a834511dc1b01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x17a06afe0>)
(PPO pid=7002) ValueError: The two structures don't have the same nested structure.
(PPO pid=7002) 
(PPO pid=7002) First structure: type=ndarray str=

2024-05-13 00:09:17,326	ERROR tune_controller.py:1331 -- Trial task failed for trial PPO_CodersStrikeBackMulti_b31d7_00000
Traceback (most recent call last):
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/_private/worker.py", line 2623, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/_private/worker.py", line 861, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(ValueError): 

In [10]:
res.get_best_result(metric="loss", mode="min")
best_checkpoint = best_result.checkpoint

# League Play based on Open Spiel Example

# Example Utils
needed to be copied. Can't Import

## Self Play Callback

In [11]:
from collections import defaultdict

import numpy as np

from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.core.rl_module.rl_module import SingleAgentRLModuleSpec


class SelfPlayCallback(DefaultCallbacks):
    def __init__(self, win_rate_threshold):
        super().__init__()
        # 0=RandomPolicy, 1=1st main policy snapshot,
        # 2=2nd main policy snapshot, etc..
        self.current_opponent = 0

        self.win_rate_threshold = win_rate_threshold

        # Report the matchup counters (who played against whom?).
        self._matching_stats = defaultdict(int)

    def on_train_result(self, *, algorithm, metrics_logger=None, result, **kwargs):
        # Get the win rate for the train batch.
        # Note that normally, one should set up a proper evaluation config,
        # such that evaluation always happens on the already updated policy,
        # instead of on the already used train_batch.
        main_rew = result["hist_stats"].pop("policy_main_reward")
        opponent_rew = list(result["hist_stats"].values())[0]
        assert len(main_rew) == len(opponent_rew)
        won = 0
        for r_main, r_opponent in zip(main_rew, opponent_rew):
            if r_main > r_opponent:
                won += 1
        win_rate = won / len(main_rew)
        result["win_rate"] = win_rate
        print(f"Iter={algorithm.iteration} win-rate={win_rate} -> ", end="")
        # If win rate is good -> Snapshot current policy and play against
        # it next, keeping the snapshot fixed and only improving the "main"
        # policy.
        if win_rate > self.win_rate_threshold:
            self.current_opponent += 1
            new_module_id = f"main_v{self.current_opponent}"
            print(f"adding new opponent to the mix ({new_module_id}).")

            # Re-define the mapping function, such that "main" is forced
            # to play against any of the previously played modules
            # (excluding "random").
            def agent_to_module_mapping_fn(agent_id, episode, **kwargs):
                # agent_id = [0|1] -> policy depends on episode ID
                # This way, we make sure that both modules sometimes play
                # (start player) and sometimes agent1 (player to move 2nd).
                opponent = "main_v{}".format(
                    np.random.choice(list(range(1, self.current_opponent + 1)))
                )
                if hash(episode.id_) % 2 == agent_id:
                    self._matching_stats[("main", opponent)] += 1
                    return "main"
                else:
                    return opponent

            main_module = algorithm.get_module("main")
            algorithm.add_module(
                module_id=new_module_id,
                module_spec=SingleAgentRLModuleSpec.from_module(main_module),
                module_state=main_module.get_state(),
                new_agent_to_module_mapping_fn=agent_to_module_mapping_fn,
            )
        else:
            print("not good enough; will keep learning ...")

        # +2 = main + random
        result["league_size"] = self.current_opponent + 2

        print(f"Matchups:\n{self._matching_stats}")

## Self Play Callback OldAPI

In [12]:
import numpy as np

from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.utils.deprecation import Deprecated


@Deprecated(help="Use the example for the new RLlib API stack", error=False)
class SelfPlayCallbackOldAPIStack(DefaultCallbacks):
    def __init__(self, win_rate_threshold):
        super().__init__()
        # 0=RandomPolicy, 1=1st main policy snapshot,
        # 2=2nd main policy snapshot, etc..
        self.current_opponent = 0

        self.win_rate_threshold = win_rate_threshold

    def on_train_result(self, *, algorithm, result, **kwargs):
        # Get the win rate for the train batch.
        # Note that normally, one should set up a proper evaluation config,
        # such that evaluation always happens on the already updated policy,
        # instead of on the already used train_batch.
        main_rew = result["hist_stats"].pop("policy_main_reward")
        opponent_rew = list(result["hist_stats"].values())[0]
        assert len(main_rew) == len(opponent_rew)
        won = 0
        for r_main, r_opponent in zip(main_rew, opponent_rew):
            if r_main > r_opponent:
                won += 1
        win_rate = won / len(main_rew)
        result["win_rate"] = win_rate
        print(f"Iter={algorithm.iteration} win-rate={win_rate} -> ", end="")
        # If win rate is good -> Snapshot current policy and play against
        # it next, keeping the snapshot fixed and only improving the "main"
        # policy.
        if win_rate > self.win_rate_threshold:
            self.current_opponent += 1
            new_pol_id = f"main_v{self.current_opponent}"
            print(f"adding new opponent to the mix ({new_pol_id}).")

            # Re-define the mapping function, such that "main" is forced
            # to play against any of the previously played policies
            # (excluding "random").
            def policy_mapping_fn(agent_id, episode, worker, **kwargs):
                # agent_id = [0|1] -> policy depends on episode ID
                # This way, we make sure that both policies sometimes play
                # (start player) and sometimes agent1 (player to move 2nd).
                return (
                    "main"
                    if episode.episode_id % 2 == agent_id
                    else "main_v{}".format(
                        np.random.choice(list(range(1, self.current_opponent + 1)))
                    )
                )

            main_policy = algorithm.get_policy("main")
            new_policy = algorithm.add_policy(
                policy_id=new_pol_id,
                policy_cls=type(main_policy),
                policy_mapping_fn=policy_mapping_fn,
            )

            # Set the weights of the new policy to the main policy.
            # We'll keep training the main policy, whereas `new_pol_id` will
            # remain fixed.
            main_state = main_policy.get_state()
            new_policy.set_state(main_state)
            # We need to sync the just copied local weights (from main policy)
            # to all the remote workers as well.
            algorithm.workers.sync_weights()
        else:
            print("not good enough; will keep learning ...")

        # +2 = main + random
        result["league_size"] = self.current_opponent + 2

## Self Play League Based Callback

In [13]:
from collections import defaultdict
from pprint import pprint
import re

import numpy as np

from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.core.rl_module.rl_module import SingleAgentRLModuleSpec


class SelfPlayLeagueBasedCallback(DefaultCallbacks):
    def __init__(self, win_rate_threshold):
        super().__init__()
        # All policies in the league.
        self.main_policies = {"main", "main_0"}
        self.main_exploiters = {"main_exploiter_0", "main_exploiter_1"}
        self.league_exploiters = {"league_exploiter_0", "league_exploiter_1"}
        # Set of currently trainable policies in the league.
        self.trainable_policies = {"main"}
        # Set of currently non-trainable (frozen) policies in the league.
        self.non_trainable_policies = {
            "main_0",
            "league_exploiter_0",
            "main_exploiter_0",
        }
        # The win-rate value reaching of which leads to a new module being added
        # to the leage (frozen copy of main).
        self.win_rate_threshold = win_rate_threshold
        # Store the win rates for league overview printouts.
        self.win_rates = {}

        # Report the matchup counters (who played against whom?).
        self._matching_stats = defaultdict(int)

    def on_train_result(self, *, algorithm, metrics_logger=None, result, **kwargs):
        local_worker = algorithm.workers.local_worker()

        # Avoid `self` being pickled into the remote function below.
        _trainable_policies = self.trainable_policies

        # Get the win rate for the train batch.
        # Note that normally, one should set up a proper evaluation config,
        # such that evaluation always happens on the already updated policy,
        # instead of on the already used train_batch.
        league_changed = False
        for module_id, rew in result["hist_stats"].items():
            mo = re.match("^policy_(.+)_reward$", module_id)
            if mo is None:
                continue
            module_id = mo.group(1)

            # Calculate this policy's win rate.
            won = 0
            for r in rew:
                if r > 0.0:  # win = 1.0; loss = -1.0
                    won += 1
            win_rate = won / len(rew)
            self.win_rates[module_id] = win_rate

            # Policy is frozen; ignore.
            if module_id in self.non_trainable_policies:
                continue

            print(
                f"Iter={algorithm.iteration} {module_id}'s " f"win-rate={win_rate} -> ",
                end="",
            )

            # If win rate is good -> Snapshot current policy and decide,
            # whether to freeze the copy or not.
            if win_rate > self.win_rate_threshold:
                is_main = re.match("^main(_\\d+)?$", module_id)
                initializing_exploiters = False

                # First time, main manages a decent win-rate against random:
                # Add league_exploiter_1 and main_exploiter_1 as trainables to the mix.
                if is_main and len(self.trainable_policies) == 1:
                    initializing_exploiters = True
                    self.trainable_policies.add("league_exploiter_1")
                    self.trainable_policies.add("main_exploiter_1")
                # If main manages to win (above threshold) against the entire league
                # -> increase the league by another frozen copy of main,
                # main-exploiters or league-exploiters.
                else:
                    keep_training = (
                        False
                        if is_main
                        else np.random.choice([True, False], p=[0.3, 0.7])
                    )
                    if module_id in self.main_policies:
                        new_mod_id = re.sub(
                            "(main)(_\\d+)?$",
                            f"\\1_{len(self.main_policies) - 1}",
                            module_id,
                        )
                        self.main_policies.add(new_mod_id)
                    elif module_id in self.main_exploiters:
                        new_mod_id = re.sub(
                            "_\\d+$", f"_{len(self.main_exploiters)}", module_id
                        )
                        self.main_exploiters.add(new_mod_id)
                    else:
                        new_mod_id = re.sub(
                            "_\\d+$", f"_{len(self.league_exploiters)}", module_id
                        )
                        self.league_exploiters.add(new_mod_id)

                    if keep_training:
                        self.trainable_policies.add(new_mod_id)
                    else:
                        self.non_trainable_policies.add(new_mod_id)

                    print(f"adding new opponents to the mix ({new_mod_id}).")

                # Update our mapping function accordingly.
                def agent_to_module_mapping_fn(agent_id, episode, **kwargs):
                    # Pick, whether this is ...
                    type_ = np.random.choice([1, 2])

                    # 1) League exploiter vs any other.
                    if type_ == 1:
                        league_exploiter = "league_exploiter_" + str(
                            np.random.choice(list(range(len(self.league_exploiters))))
                        )
                        # This league exploiter is frozen: Play against a
                        # trainable policy.
                        if league_exploiter not in self.trainable_policies:
                            opponent = np.random.choice(list(self.trainable_policies))
                        # League exploiter is trainable: Play against any other
                        # non-trainable policy.
                        else:
                            opponent = np.random.choice(
                                list(self.non_trainable_policies)
                            )

                        # Only record match stats once per match.
                        if hash(episode.id_) % 2 == agent_id:
                            self._matching_stats[(league_exploiter, opponent)] += 1
                            return league_exploiter
                        else:
                            return opponent

                    # 2) Main exploiter vs main.
                    else:
                        main_exploiter = "main_exploiter_" + str(
                            np.random.choice(list(range(len(self.main_exploiters))))
                        )
                        # Main exploiter is frozen: Play against the main
                        # policy.
                        if main_exploiter not in self.trainable_policies:
                            main = "main"
                        # Main exploiter is trainable: Play against any
                        # frozen main.
                        else:
                            main = np.random.choice(list(self.main_policies - {"main"}))

                        # Only record match stats once per match.
                        if hash(episode.id_) % 2 == agent_id:
                            self._matching_stats[(main_exploiter, main)] += 1
                            return main_exploiter
                        else:
                            return main

                marl_module = local_worker.module
                main_module = marl_module["main"]

                # Set the weights of the new polic(y/ies).
                if initializing_exploiters:
                    main_state = main_module.get_state()
                    marl_module["main_0"].set_state(main_state)
                    marl_module["league_exploiter_1"].set_state(main_state)
                    marl_module["main_exploiter_1"].set_state(main_state)
                    # We need to sync the just copied local weights to all the
                    # remote workers and remote Learner workers as well.
                    algorithm.workers.sync_weights(
                        policies=["main_0", "league_exploiter_1", "main_exploiter_1"]
                    )
                    algorithm.learner_group.set_weights(marl_module.get_state())
                else:
                    algorithm.add_module(
                        module_id=new_mod_id,
                        module_spec=SingleAgentRLModuleSpec.from_module(main_module),
                        module_state=marl_module[module_id].get_state(),
                    )

                algorithm.workers.foreach_worker(
                    lambda env_runner: env_runner.config.multi_agent(
                        policy_mapping_fn=agent_to_module_mapping_fn,
                        # This setting doesn't really matter for EnvRunners (no
                        # training going on there, but we'll update this as well
                        # here for good measure).
                        policies_to_train=_trainable_policies,
                    ),
                    local_worker=True,
                )
                # Set all Learner workers' should_module_be_updated to the new
                # value.
                algorithm.learner_group.foreach_learner(
                    lambda learner: learner.config.multi_agent(
                        policies_to_train=_trainable_policies,
                    )
                )
                league_changed = True
            else:
                print("not good enough; will keep learning ...")

        # Add current league size to results dict.
        result["league_size"] = len(self.non_trainable_policies) + len(
            self.trainable_policies
        )

        if league_changed:
            self._print_league()

    def _print_league(self):
        print("--- League ---")
        print("Matchups:")
        pprint(self._matching_stats)
        print("Trainable policies (win-rates):")
        for p in sorted(self.trainable_policies):
            wr = self.win_rates[p] if p in self.win_rates else 0.0
            print(f"\t{p}: {wr}")
        print("Frozen policies:")
        for p in sorted(self.non_trainable_policies):
            wr = self.win_rates[p] if p in self.win_rates else 0.0
            print(f"\t{p}: {wr}")
        print()

## SelfPlayLeagueBasedCallbackOldAPIStack

In [14]:
import re

import numpy as np

from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.utils.deprecation import Deprecated


@Deprecated(help="Use the example for the new RLlib API stack", error=False)
class SelfPlayLeagueBasedCallbackOldAPIStack(DefaultCallbacks):
    def __init__(self, win_rate_threshold):
        super().__init__()
        # All policies in the league.
        self.main_policies = {"main", "main_0"}
        self.main_exploiters = {"main_exploiter_0", "main_exploiter_1"}
        self.league_exploiters = {"league_exploiter_0", "league_exploiter_1"}
        # Set of currently trainable policies in the league.
        self.trainable_policies = {"main"}
        # Set of currently non-trainable (frozen) policies in the league.
        self.non_trainable_policies = {
            "main_0",
            "league_exploiter_0",
            "main_exploiter_0",
        }
        # The win-rate value reaching of which leads to a new module being added
        # to the leage (frozen copy of main).
        self.win_rate_threshold = win_rate_threshold
        # Store the win rates for league overview printouts.
        self.win_rates = {}

    def on_train_result(self, *, algorithm, result, **kwargs):
        # Avoid `self` being pickled into the remote function below.
        _trainable_policies = self.trainable_policies

        # Get the win rate for the train batch.
        # Note that normally, one should set up a proper evaluation config,
        # such that evaluation always happens on the already updated policy,
        # instead of on the already used train_batch.
        for policy_id, rew in result["hist_stats"].items():
            mo = re.match("^policy_(.+)_reward$", policy_id)
            if mo is None:
                continue
            policy_id = mo.group(1)

            # Calculate this policy's win rate.
            won = 0
            for r in rew:
                if r > 0.0:  # win = 1.0; loss = -1.0
                    won += 1
            win_rate = won / len(rew)
            self.win_rates[policy_id] = win_rate

            # Policy is frozen; ignore.
            if policy_id in self.non_trainable_policies:
                continue

            print(
                f"Iter={algorithm.iteration} {policy_id}'s " f"win-rate={win_rate} -> ",
                end="",
            )

            # If win rate is good -> Snapshot current policy and decide,
            # whether to freeze the copy or not.
            if win_rate > self.win_rate_threshold:
                is_main = re.match("^main(_\\d+)?$", policy_id)
                initializing_exploiters = False

                # First time, main manages a decent win-rate against random:
                # Add league_exploiter_0 and main_exploiter_0 to the mix.
                if is_main and len(self.trainable_policies) == 1:
                    initializing_exploiters = True
                    self.trainable_policies.add("league_exploiter_0")
                    self.trainable_policies.add("main_exploiter_0")
                else:
                    keep_training = (
                        False
                        if is_main
                        else np.random.choice([True, False], p=[0.3, 0.7])
                    )
                    if policy_id in self.main_policies:
                        new_pol_id = re.sub(
                            "_\\d+$", f"_{len(self.main_policies) - 1}", policy_id
                        )
                        self.main_policies.add(new_pol_id)
                    elif policy_id in self.main_exploiters:
                        new_pol_id = re.sub(
                            "_\\d+$", f"_{len(self.main_exploiters)}", policy_id
                        )
                        self.main_exploiters.add(new_pol_id)
                    else:
                        new_pol_id = re.sub(
                            "_\\d+$", f"_{len(self.league_exploiters)}", policy_id
                        )
                        self.league_exploiters.add(new_pol_id)

                    if keep_training:
                        self.trainable_policies.add(new_pol_id)
                    else:
                        self.non_trainable_policies.add(new_pol_id)

                    print(f"adding new opponents to the mix ({new_pol_id}).")

                # Update our mapping function accordingly.
                def policy_mapping_fn(agent_id, episode, worker=None, **kwargs):
                    # Pick, whether this is ...
                    type_ = np.random.choice([1, 2])

                    # 1) League exploiter vs any other.
                    if type_ == 1:
                        league_exploiter = "league_exploiter_" + str(
                            np.random.choice(list(range(len(self.league_exploiters))))
                        )
                        # This league exploiter is frozen: Play against a
                        # trainable policy.
                        if league_exploiter not in self.trainable_policies:
                            opponent = np.random.choice(list(self.trainable_policies))
                        # League exploiter is trainable: Play against any other
                        # non-trainable policy.
                        else:
                            opponent = np.random.choice(
                                list(self.non_trainable_policies)
                            )
                        print(f"{league_exploiter} vs {opponent}")
                        return (
                            league_exploiter
                            if episode.episode_id % 2 == agent_id
                            else opponent
                        )

                    # 2) Main exploiter vs main.
                    else:
                        main_exploiter = "main_exploiter_" + str(
                            np.random.choice(list(range(len(self.main_exploiters))))
                        )
                        # Main exploiter is frozen: Play against the main
                        # policy.
                        if main_exploiter not in self.trainable_policies:
                            main = "main"
                        # Main exploiter is trainable: Play against any
                        # frozen main.
                        else:
                            main = np.random.choice(list(self.main_policies - {"main"}))
                        # print(f"{main_exploiter} vs {main}")
                        return (
                            main_exploiter
                            if episode.episode_id % 2 == agent_id
                            else main
                        )

                # Set the weights of the new polic(y/ies).
                if initializing_exploiters:
                    main_state = algorithm.get_policy("main").get_state()
                    pol_map = algorithm.workers.local_worker().policy_map
                    pol_map["main_0"].set_state(main_state)
                    pol_map["league_exploiter_1"].set_state(main_state)
                    pol_map["main_exploiter_1"].set_state(main_state)
                    # We need to sync the just copied local weights to all the
                    # remote workers as well.
                    algorithm.workers.sync_weights(
                        policies=["main_0", "league_exploiter_1", "main_exploiter_1"]
                    )

                    def _set(worker):
                        worker.set_policy_mapping_fn(policy_mapping_fn)
                        worker.set_is_policy_to_train(_trainable_policies)

                    algorithm.workers.foreach_worker(_set)
                else:
                    new_policy = algorithm.add_policy(
                        policy_id=new_pol_id,
                        policy_cls=type(algorithm.get_policy(policy_id)),
                        policy_mapping_fn=policy_mapping_fn,
                        policies_to_train=self.trainable_policies,
                    )
                    main_state = algorithm.get_policy(policy_id).get_state()
                    new_policy.set_state(main_state)
                    # We need to sync the just copied local weights to all the
                    # remote workers as well.
                    algorithm.workers.sync_weights(policies=[new_pol_id])

                self._print_league()

            else:
                print("not good enough; will keep learning ...")

    def _print_league(self):
        print("--- League ---")
        print("Trainable policies (win-rates):")
        for p in sorted(self.trainable_policies):
            wr = self.win_rates[p] if p in self.win_rates else 0.0
            print(f"\t{p}: {wr}")
        print("Frozen policies:")
        for p in sorted(self.non_trainable_policies):
            wr = self.win_rates[p] if p in self.win_rates else 0.0
            print(f"\t{p}: {wr}")
        print()

# Random Policy Modules

In [15]:
import pathlib
from typing import Mapping, Any

import gymnasium as gym
import numpy as np
import tree  # pip install dm_tree

from ray.rllib.core.columns import Columns
from ray.rllib.core.rl_module import RLModule
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.utils.annotations import override
from ray.rllib.utils.spaces.space_utils import batch as batch_func


class RandomRLModule(RLModule):
    @override(RLModule)
    def _forward_inference(self, batch, **kwargs):
        return self._random_forward(batch, **kwargs)

    @override(RLModule)
    def _forward_exploration(self, batch, **kwargs):
        return self._random_forward(batch, **kwargs)

    @override(RLModule)
    def _forward_train(self, *args, **kwargs):
        # RandomRLModule should always be configured as non-trainable.
        # To do so, set in your config:
        # `config.multi_agent(policies_to_train=[list of ModuleIDs to be trained,
        # NOT including the ModuleID of this RLModule])`
        raise NotImplementedError("Random RLModule: Should not be trained!")

    @override(RLModule)
    def output_specs_inference(self):
        return [SampleBatch.ACTIONS]

    @override(RLModule)
    def output_specs_exploration(self):
        return [SampleBatch.ACTIONS]

    def _random_forward(self, batch, **kwargs):
        obs_batch_size = len(tree.flatten(batch[SampleBatch.OBS])[0])
        actions = batch_func(
            [self.config.action_space.sample() for _ in range(obs_batch_size)]
        )
        return {SampleBatch.ACTIONS: actions}

    def _module_state_file_name(self) -> pathlib.Path:
        return pathlib.Path("random_rl_module_dummy_state")

    def compile(self, *args, **kwargs):
        """Dummy method for compatibility with TorchRLModule.

        This is hit when RolloutWorker tries to compile TorchRLModule."""
        pass

    @classmethod
    def from_model_config(
        cls,
        observation_space: gym.Space,
        action_space: gym.Space,
        *,
        model_config_dict: Mapping[str, Any],
    ) -> "RLModule":
        return cls(action_space)


class StatefulRandomRLModule(RandomRLModule):
    """A stateful RLModule that returns STATE_OUT from its forward methods.

    - Implements the `get_initial_state` method (returning a all-zeros dummy state).
    - Returns a dummy state under the `Columns.STATE_OUT` from its forward methods.
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._internal_state_space = gym.spaces.Box(-1.0, 1.0, (1,))

    @override(RLModule)
    def get_initial_state(self):
        return {
            "state": np.zeros_like([self._internal_state_space.sample()]),
        }

    def _random_forward(self, batch, **kwargs):
        batch = super()._random_forward(batch, **kwargs)
        batch[Columns.STATE_OUT] = {
            "state": batch_func(
                [
                    self._internal_state_space.sample()
                    for _ in range(len(batch[Columns.ACTIONS]))
                ]
            ),
        }
        return batch

In [16]:
from gymnasium.spaces import Box
import numpy as np
import random
import tree  # pip install dm_tree
from typing import (
    List,
    Optional,
    Union,
)

from ray.rllib.policy.policy import Policy
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.utils.annotations import override
from ray.rllib.utils.typing import ModelWeights, TensorStructType, TensorType


class RandomPolicy(Policy):
    """Hand-coded policy that returns random actions."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # Whether for compute_actions, the bounds given in action_space
        # should be ignored (default: False). This is to test action-clipping
        # and any Env's reaction to bounds breaches.
        if self.config.get("ignore_action_bounds", False) and isinstance(
            self.action_space, Box
        ):
            self.action_space_for_sampling = Box(
                -float("inf"),
                float("inf"),
                shape=self.action_space.shape,
                dtype=self.action_space.dtype,
            )
        else:
            self.action_space_for_sampling = self.action_space

    @override(Policy)
    def init_view_requirements(self):
        super().init_view_requirements()
        # Disable for_training and action attributes for SampleBatch.INFOS column
        # since it can not be properly batched.
        vr = self.view_requirements[SampleBatch.INFOS]
        vr.used_for_training = False
        vr.used_for_compute_actions = False

    @override(Policy)
    def compute_actions(
        self,
        obs_batch: Union[List[TensorStructType], TensorStructType],
        state_batches: Optional[List[TensorType]] = None,
        prev_action_batch: Union[List[TensorStructType], TensorStructType] = None,
        prev_reward_batch: Union[List[TensorStructType], TensorStructType] = None,
        **kwargs,
    ):
        # Alternatively, a numpy array would work here as well.
        # e.g.: np.array([random.choice([0, 1])] * len(obs_batch))
        obs_batch_size = len(tree.flatten(obs_batch)[0])
        return (
            [self.action_space_for_sampling.sample() for _ in range(obs_batch_size)],
            [],
            {},
        )

    @override(Policy)
    def learn_on_batch(self, samples):
        """No learning."""
        return {}

    @override(Policy)
    def compute_log_likelihoods(
        self,
        actions,
        obs_batch,
        state_batches=None,
        prev_action_batch=None,
        prev_reward_batch=None,
        **kwargs,
    ):
        return np.array([random.random()] * len(obs_batch))

    @override(Policy)
    def get_weights(self) -> ModelWeights:
        """No weights to save."""
        return {}

    @override(Policy)
    def set_weights(self, weights: ModelWeights) -> None:
        """No weights to set."""
        pass

    @override(Policy)
    def _get_dummy_batch_from_view_requirements(self, batch_size: int = 1):
        return SampleBatch(
            {
                SampleBatch.OBS: tree.map_structure(
                    lambda s: s[None], self.observation_space.sample()
                ),
            }
        )

# Self Play

In [17]:
import functools

import numpy as np

import ray
from ray.rllib.core.rl_module.marl_module import MultiAgentRLModuleSpec
from ray.rllib.core.rl_module.rl_module import SingleAgentRLModuleSpec
from ray.rllib.env.utils import try_import_pyspiel, try_import_open_spiel
from ray.rllib.env.wrappers.open_spiel import OpenSpielEnv
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

open_spiel = try_import_open_spiel(error=True)
pyspiel = try_import_pyspiel(error=True)

# Import after try_import_open_spiel, so we can error out with hints
from open_spiel.python.rl_environment import Environment  # noqa: E4

In [18]:

def _get_multi_agent():
    names = {
        # Our main policy, we'd like to optimize.
        "main",
        # First frozen version of main (after we reach n% win-rate).
        "main_0",
        # Initial main exploiters (one random, one trainable).
        "main_exploiter_0",
        "main_exploiter_1",
        # Initial league exploiters (one random, one trainable).
        "league_exploiter_0",
        "league_exploiter_1",
    }
    if args.enable_new_api_stack:
        policies = names
        spec = {
            mid: SingleAgentRLModuleSpec(
                module_class=(
                    RandomRLModule
                    if mid in ["main_exploiter_0", "league_exploiter_0"]
                    else None
                )
            )
            for mid in names
        }
    else:
        policies = {
            mid: PolicySpec(
                policy_class=(
                    RandomPolicy
                    if mid in ["main_exploiter_0", "league_exploiter_0"]
                    else None
                )
            )
            for mid in names
        }
        spec = None
    return {"policies": policies, "spec": spec}

In [ ]:
config = (
    get_trainable_cls("PPO")
    .get_default_config()
    # Use new API stack ...
    .api_stack(
        enable_rl_module_and_learner=args.enable_new_api_stack,
        enable_env_runner_and_connector_v2=args.enable_new_api_stack,
    )
    .environment("open_spiel_env")
    .framework(args.framework)
    # Set up the main piece in this experiment: The league-bases self-play
    # callback, which controls adding new policies/Modules to the league and
    # properly matching the different policies in the league with each other.
    .callbacks(
        functools.partial(
            SelfPlayLeagueBasedCallback
            if args.enable_new_api_stack
            else SelfPlayLeagueBasedCallbackOldAPIStack,
            win_rate_threshold=args.win_rate_threshold,
        )
    )
    .env_runners(
        num_env_runners=args.num_env_runners,
        num_envs_per_env_runner=1 if args.enable_new_api_stack else 5,
    )
    .resources(
        num_learner_workers=args.num_gpus,
        num_gpus_per_learner_worker=1 if args.num_gpus else 0,
        num_cpus_for_local_worker=1,
    )
    .training(
        num_sgd_iter=20,
        model=dict(
            **({"uses_new_env_runners": True} if args.enable_new_api_stack else {}),
        ),
    )
    .multi_agent(
        # Initial policy map: All PPO. This will be expanded
        # to more policy snapshots. This is done in the
        # custom callback defined above (`LeagueBasedSelfPlayCallback`).
        policies=_get_multi_agent()["policies"],
        policy_mapping_fn=(
            agent_to_module_mapping_fn
            if args.enable_new_api_stack
            else policy_mapping_fn
        ),
        # At first, only train main_0 (until good enough to win against
        # random).
        policies_to_train=["main"],
    )
    .rl_module(
        rl_module_spec=MultiAgentRLModuleSpec(
            module_specs=_get_multi_agent()["spec"]
        ),
    )
)

# Play against the above trained strategy

In [19]:
import random 

class CSBSelfPlay(MultiAgentEnv):
    def __init__(self):
        # These are all the possible agents (main can play as team 0 and 1.  Old can only play team 1)
        self.env = CodersStrikeBackMulti(dt=1)
        self._agent_ids = []
        for team in self.env.teams:
            for car in range(self.env.num_racers_per_team):
                self._agent_ids.append(f'main_{team}_{car}')
                if team > 0:
                    self._agent_ids.append(f'old_{team}_{car}')
        # Override default action space so that it covers all possible strategies
        self.observation_space = gym.spaces.Dict(make_agent_dictionary(self._agent_ids, self.env.ind_observation_space))
        self.action_space = gym.spaces.Dict(make_agent_dictionary(self._agent_ids, self.env.ind_action_space))
    
    def reset(self, **kwargs):
        self.create_competitor_pool()
        return self.env.reset(**kwargs)

    def create_competitor_pool(self):
        selected_ids = []
        policies = ['main'] + random.choices(['main','old'], weights=[0.8,0.2], k=self.env.num_teams-1)
        for team in range(self.env.num_teams):
            for car in range(self.env.num_racers_per_team):
                selected_ids.append(f'{policies[team]}_{team}_{car}')
        self.env._agent_ids = selected_ids
        for i, racer in enumerate(self.env.racers):
            racer.aid = selected_ids[i]

    def step(self, **kwargs):
        return self.env.step(**kwargs)

In [20]:
env = CSBSelfPlay()

In [21]:
env._agent_ids

['main_0_0', 'main_0_1', 'main_1_0', 'old_1_0', 'main_1_1', 'old_1_1']

In [22]:
env.reset()

({'main_0_0': array([ 3.00000000e+00, -1.66278628e+00,  2.82971635e+03,  8.56933191e+03,
          0.00000000e+00,  0.00000000e+00,  2.17600000e+03,  1.48300000e+03,
          1.40860000e+04,  8.15900000e+03,  4.52200000e+03,  7.95600000e+03,
          2.17600000e+03,  1.48300000e+03,  1.40860000e+04,  8.15900000e+03]),
  'main_0_1': array([ 3.00000000e+00, -1.83157400e+00,  3.95790545e+03,  8.16044397e+03,
          0.00000000e+00,  0.00000000e+00,  2.17600000e+03,  1.48300000e+03,
          1.40860000e+04,  8.15900000e+03,  4.52200000e+03,  7.95600000e+03,
          2.17600000e+03,  1.48300000e+03,  1.40860000e+04,  8.15900000e+03]),
  'main_1_0': array([ 3.00000000e+00, -2.00542632e+00,  5.08609455e+03,  7.75155603e+03,
          0.00000000e+00,  0.00000000e+00,  2.17600000e+03,  1.48300000e+03,
          1.40860000e+04,  8.15900000e+03,  4.52200000e+03,  7.95600000e+03,
          2.17600000e+03,  1.48300000e+03,  1.40860000e+04,  8.15900000e+03]),
  'main_1_1': array([ 3.00000000e+